In [28]:
import os
import shutil
from glob import glob
import warnings

import earthpy as et
import earthpy.earthexplorer as etee
import earthpy.spatial as es
import geopandas as gpd
import geoviews as gv
from holoviews import opts
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd 
import pyogrio
import rioxarray as rxr
import rioxarray.merge as rxrmerge

import zipfile
from zipfile import BadZipFile

data_dir= os.path.join(et.io.HOME, et.io.DATA_NAME)
den_dir = os.path.join(data_dir, 'denver-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'denver-green-space', 'processed')

for a_dir in [den_dir, ndvi_dir]:
        if not os.path.exists(a_dir):
                os.makedirs(a_dir)


warnings.simplefilter('ignore')

In [29]:
den_path = os.path.join(den_dir, 'denver-neighborhoods.shp')
if not os.path.exists(den_path):
    den_url = ("https://www.denvergov.org/media/gis/DataCatalog/statistical_neighborhoods/shape/statistical_neighborhoods.zip")
    gpd.read_file(den_url).to_file(den_path)

gpd.read_file(den_path)
den_gdf = gpd.read_file(den_path).set_index('NBHD_NAME')

neigh_gdf = (den_gdf.loc[['Lowry Field', 'Fort Logan','University Park']])
neigh_gdf

,NBHD_ID,TYPOLOGY,NOTES,geometry
NBHD_NAME,,,,
Lowry Field,42,None,None,"POLYGON ((-104.88470 39.73285, -104.88471 39.7..."
Fort Logan,27,None,None,"POLYGON ((-105.03462 39.65328, -105.03463 39.6..."
University Park,66,None,None,"POLYGON ((-104.94069 39.68296, -104.94069 39.6..."


In [30]:
gpd.read_file(den_path)

,NBHD_ID,NBHD_NAME,TYPOLOGY,NOTES,geometry
0,56,Sloan Lake,None,None,"POLYGON ((-105.02523 39.75848, -105.02524 39.7..."
1,39,Jefferson Park,None,None,"POLYGON ((-105.00949 39.75393, -105.00976 39.7..."
2,14,City Park,None,None,"POLYGON ((-104.94062 39.75104, -104.94063 39.7..."
3,42,Lowry Field,None,None,"POLYGON ((-104.88470 39.73285, -104.88471 39.7..."
4,27,Fort Logan,None,None,"POLYGON ((-105.03462 39.65328, -105.03463 39.6..."
...,...,...,...,...,...
73,74,West Colfax,None,Removed NEST Status May 2023,"POLYGON ((-105.02521 39.74636, -105.02521 39.7..."
74,33,Hampden South,None,None,"POLYGON ((-104.90205 39.62443, -104.90237 39.6..."
75,67,Valverde,NEST,None,"POLYGON ((-105.01577 39.72550, -105.01582 39.7..."
76,5,Barnum West,NEST,None,"POLYGON ((-105.03945 39.72571, -105.03945 39.7..."


In [31]:
(gv.tile_sources.OSM 
 * neigh_gdf.to_crs(3857).hvplot(
     line_color = 'red',fill_color = None, 
     line_width = 3, xaxis = None, yaxis = None))

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [x,y]

In [32]:
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date
    Downloads data from the National Agricultural Imagery Program (NAIP)
    given a spatial and temporal extent.

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data 
      directory.
    """

    print(f'\nNeighborhood name: {name}')
    bbox = etee.BBox(*geometry.bounds)
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label=name.lower().replace(' ','-'), 
        bbox=bbox,
        start= start, 
        end= end ,
        store_credential=True)
    naip_downloader.submit_download_request()
  
    # naip_downloader.download(override=True)
    
   
    return naip_downloader

ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')
if os.path.exists(ndvi_stats_path):
  print('reading in NDVI stats file ...')
  ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col='neighborhood')
else:
   print('NDVI stats file DNE')
   ndvi_stats_df = pd.DataFrame()
   
for neighborhood_name, details in neigh_gdf.iterrows():
    if neighborhood_name in ndvi_stats_df.index:
      print('neighborhood stats have already been calculated. skipping')
      continue
    downloader = download_neighborhood_data(
       neighborhood_name, details.geometry,'2019-01-01','2019-12-31')
    print(neighborhood_name)
    print(details.geometry.bounds)


NDVI stats file DNE

Neighborhood name: Lowry Field
Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...


Found 2 scenes
4 products found.
Downloads staging...
Lowry Field
(-104.9127812938757, 39.70691198246254, -104.87064364468677, 39.73289505192241)

Neighborhood name: Fort Logan
Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...
Found 4 scenes
8 products found.
Downloads staging...
Fort Logan
(-105.07691086355908, 39.62683574735013, -105.02540068629685, 39.654408633589696)

Neighborhood name: University Park
Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...
Found 1 scenes
2 products found.
Downloads staging...
University Park
(-104.95944712952661, 39.66694072614939, -104.9406795762983, 39.68473922085474)


In [33]:
# import requests
# downloads = downloader.wait_for_available_downloads()



# for dl in downloads["available"]:
#     print(dl['url'])
# #     response_1 = requests.post('https://m2m.cr.usgs.gov/api/api/json/stable/download-remove?downloadId={}'.format(download['downloadId']))
# #     print(response_1.content)
# downloads

In [34]:
# %%bash
# wget "https://dds.cr.usgs.gov/download/eyJpZCI6NTEwNzE2ODk2LCJjb250YWN0SWQiOjI2OTQxNzAyfQ==" -o /home/jovyan/earth-analytics/data/lowry-field/M_3910418_NW_13_1_20090630.tif

In [35]:
def load_and_merge_arrays(name):
    """
    Load in and merge downloaded arrays

    Parameters
    ==========
    name : str
      The name used to label the download
    
    Returns
    =======
    Merged Data Array : rxr.DataArray
        DataArray with merged data
    """

    print(f'\nNeighborhood name: {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME,
        name.lower().replace(' ', '-'))
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked = True) for tp in tif_paths]
    merged_da = rxrmerge.merge_arrays(das)
    return merged_da

# run to test function
# merged_da = load_and_merge_arrays('Jefferson park')
# merged_da
 

In [36]:
def load_and_merge_arrays(name):
    """
    Load in and merge downloaded arrays

    Parameters
    ==========
    name : str
      The name used to label the download
    
    Returns
    =======
    Merged Data Array : rxr.DataArray
        DataArray with merged data
    """

    print(f'\nNeighborhood name: {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME,
        name.lower().replace(' ', '-'))
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked = True) for tp in tif_paths]
    merged_da = rxrmerge.merge_arrays(das)
    return merged_da

# run to test function
# merged_da = load_and_merge_arrays('Jefferson Park')
# merged_da


In [37]:
def load_and_merge_arrays(name):
    """
    Load in and merge downloaded arrays

    Parameters
    ==========
    name : str
      The name used to label the download
    
    Returns
    =======
    Merged Data Array : rxr.DataArray
        DataArray with merged data
    """

    print(f'\nNeighborhood name: {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME,
        name.lower().replace(' ', '-'))
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked = True) for tp in tif_paths]
    merged_da = rxrmerge.merge_arrays(das)
    return merged_da

# run to test function
# merged_da = load_and_merge_arrays('Lincoln Park')
# merged_da
 

In [38]:
def load_and_merge_arrays(name):
    """
    Load in and merge downloaded arrays

    Parameters
    ==========
    name : str
      The name used to label the download
    
    Returns
    =======
    Merged Data Array : rxr.DataArray
        DataArray with merged data
    """

    print(f'\nNeighborhood name: {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME,
        name.lower().replace(' ', '-'))
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked = True) for tp in tif_paths]
    merged_da = rxrmerge.merge_arrays(das)
    return merged_da

# run to test function
merged_da = load_and_merge_arrays('fort logan')
merged_da
 


Neighborhood name: fort logan


IndexError: list index out of range

In [ ]:

ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col = "neighborhood")

# Initialize Holoviews extension with the desired backend
hv.extension('bokeh')  # or 'matplotlib' if you prefer

# Combine GeoPandas DataFrame with NDVI statistics
combined_gdf = den_gdf.join(ndvi_stats_df, how='left')

# Create GeoViews element
polygons = gv.Polygons(combined_gdf, vdims=['ndvi_mean'])

# Plot options
plot_opts = opts.Polygons(tools=['hover'], cmap='viridis', width=800, height=600, colorbar=True, title = 'Mean NDVI observations for Neighborhoods Across Chicago')

# Create the plot
plot = gv.tile_sources.StamenToner * polygons.opts(plot_opts)

# Show the plot
plot

In [ ]:

# def download_neighborhood_data(name, geometry, start, end):
#     """
#     Download NAIP raster for a given geometry, start date, and end date
#     Downloads data from the National Agricultural Imagery Program (NAIP)
#     given a spatial and temporal extent.

#     Parameters
#     ==========
#     name : str
#       The name used to label the download
#     geometry : shapely.POLYGON
#       The geometry to derive the download extent from. 
#       Must have a `.bounds` attribute.
#     start : str
#       The start date as 'YYYY-MM-DD'
#     end : str
#       The end date as 'YYYY-MM-DD'

#     Returns
#     =======
#     downloader : earthpy.earthexplorer.EarthExplorerDownloader
#       Object with information about the download, including the data 
#       directory.
#     """

#     print(f'\nNeighborhood name: {name}')
#     bbox = etee.BBox(*geometry.bounds)
#     naip_downloader = etee.EarthExplorerDownloader(
#         dataset="NAIP", 
#         label=name.lower().replace(' ','-'), 
#         bbox=bbox,
#         start=start, 
#         end=end,
#         store_credential=True)

#     try:
#         # naip_downloader.submit_download_request()
#         naip_downloader.wait_for_available_downloads()
#         # naip_downloader.download(override=True,wait=True)
        
#     except zipfile.BadZipFile as e:
#         print(f"Error: {e}")
#         # You can add more detailed error handling or logging here

#     return naip_downloader

# # Rest of your code...

# ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')
# if os.path.exists(ndvi_stats_path):
#     print('reading in NDVI stats file ...')
#     ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col='neighborhood')
# else:
#     print('NDVI stats file DNE')
#     ndvi_stats_df = pd.DataFrame()

# for neighborhood_name, details in neigh_gdf.iterrows():
#     if neighborhood_name in ndvi_stats_df.index:
#         print('neighborhood stats have already been calculated. skipping')
#         continue
#     downloader = download_neighborhood_data(
#         neighborhood_name, details.geometry, '2019-01-01', '2019-12-31')
    